Modeling the brusselator

In [2]:
import numpy as np
from scipy.fft import fftn, ifftn
%matplotlib inline

Brusselator:

$$\dot u = \triangle u +a - (b+1)u + u^2v$$
$$ \dot v = \triangle v + bu - u^2v$$
or in other words

$$\dot u = \triangle u f_u$$
$$ \dot v = \triangle v +f_v$$
This is in 2D. With random initial data. Do I need to worry about initial conditions? No!
I also want to be able to add a new function and a parameter

In [3]:
N = 64
dt = 0.005
fu = lambda a,b,u,v: a + (u**2)*v - (b+1)*u
fv = lambda a,b,u,v: b*u-(u**2)*v
L = 10*2*np.pi
NUM_STEPS = 1000
a = 1
b = 3
D0 = 1 
D1 = 0.1
# put only things I want to change
# don't want to change diffusion too much. At least for now.



In [4]:
def create_initial_array(num_of_nodes = N, ampl = 0.1):
    #return initial concentration of substance. Can be random, should be positive
    return np.random.rand(num_of_nodes,num_of_nodes) * ampl
# for now
def create_wavenumber_array(num_modes=N, L = L):
    # create array for wavenumbers in scipy format.
    num_modes = num_modes - num_modes%2 # make num of nodes even. Why?
    wavenum_array = np.zeros(num_modes)
    wavenum = 2*np.pi/L
    p1 = np.array([(wavenum * i) for i in range(int(num_modes/2))])
    
    p2 = np.array([wavenum * (-num_modes+n) for n in range(int(num_modes/2), num_modes)])
    wavenum_array = np.concatenate((p1,p2))
    return wavenum_array
def create_time_operator(wavenums, diffusion_coeff, timestep = dt):
    ## create a time evolution operator
    num_nodes = wavenums.size
    operator = np.empty((num_nodes,num_nodes))
    for i in range(num_nodes):
        for j in range(num_nodes):
            wavenumber = -(wavenums[i]**2+wavenums[j]**2)
            operator[i,j] = np.exp(timestep*diffusion_coeff*wavenumber)
    return operator

In [10]:
def perform_simulation(a=a,b=b, timestep = dt, fu = fu, fv = fv, N = N, NUM_STEPS=NUM_STEPS):
    
    # initialize data
    initial_data_u = create_initial_array(N)
    initial_data_v = create_initial_array(N)
    wavenums = create_wavenumber_array(N, L)

    ## initialize operators
    operator_u = create_time_operator(wavenums, D0, timestep)
    operator_v = create_time_operator(wavenums, D1, timestep)
    
    ##initialize returned data
    time = np.empty(NUM_STEPS)
    time[0] = 0
    shape = time.shape+initial_data_u.shape
    conc_u = np.empty(shape)
    conc_v = np.empty(shape)
    conc_u[0] = initial_data_u
    conc_v[0] = initial_data_v

    ## begin calculations
    for i in range(NUM_STEPS-1):
        # initialize

        u = conc_u[i]
        v = conc_v[i]
        ## apply euler scheme
        nonlin_u = u + fu(a,b,u,v)*dt
        nonlin_v = v + fv(a,b,u,v)*dt
        ## use Fourier transform
        ## NOTES: I could combine u and v into complex array.
        fft_u = fftn(nonlin_u)
        fft_v = fftn(nonlin_v)

        ## perform timestep in fourier domain
        fft_u = fft_u * operator_u
        fft_v = fft_v * operator_v

        ## go back
        u = ifftn(fft_u).real
        v = ifftn(fft_v).real

        ## record the data
        time[i+1] = time[i] + timestep
        conc_u[i+1] = u
        conc_v[i+1] = v

    return time, conc_u, conc_v


In [11]:
perform_simulation(NUM_STEPS= 10)

(array([0.   , 0.005, 0.01 , 0.015, 0.02 , 0.025, 0.03 , 0.035, 0.04 ,
        0.045]),
 array([[[0.03169719, 0.05617552, 0.09394789, ..., 0.05970021,
          0.02368842, 0.09063541],
         [0.06906924, 0.06355717, 0.04418682, ..., 0.0595951 ,
          0.09643962, 0.09895333],
         [0.04181602, 0.05745247, 0.09219168, ..., 0.0688368 ,
          0.02463971, 0.06967577],
         ...,
         [0.01927623, 0.07858434, 0.05422075, ..., 0.03339398,
          0.04101669, 0.05695536],
         [0.03035312, 0.09164877, 0.05470648, ..., 0.09739973,
          0.0863124 , 0.02715707],
         [0.03840911, 0.03155215, 0.03826562, ..., 0.08920007,
          0.0803035 , 0.09344356]],
 
        [[0.0372601 , 0.05975626, 0.09576765, ..., 0.06338203,
          0.03024537, 0.0929931 ],
         [0.07229522, 0.06707743, 0.04979118, ..., 0.06384333,
          0.09802046, 0.10143753],
         [0.04707408, 0.06109296, 0.09459846, ..., 0.07190946,
          0.03071283, 0.07264047],
         ...,

In [35]:
from matplotlib import pyplot as plt
from mpl_toolkits import mplot3d
from matplotlib import cm
# Plot u(x,y) for different time points (2D)
L = L
dt = 0.05
time, data_u, data_v = perform_simulation(NUM_STEPS= 10**3)
time = len(time)
print(time)
data = u
xs = np.arange(0.0,L,L/N)
ys = np.arange(0.0,L,L/N)

X,Y = np.meshgrid(xs,ys)

from matplotlib import animation, rc
from IPython.display import HTML

fig = plt.figure(figsize=(9.0,3.5))
ax1 = fig.add_subplot(231)
ax2 = fig.add_subplot(232)
ax3 = fig.add_subplot(233)

ax1.pcolormesh(X, Y, data_u[0], shading='auto')
ax2.pcolormesh(X, Y, data_u[time//2], shading='auto')
ax3.pcolormesh(X, Y, data_u[time-1], shading='auto')
ax4 = fig.add_subplot(234)
ax5 = fig.add_subplot(235)
ax6 = fig.add_subplot(236)

ax4.pcolormesh(X, Y, data_v[0], shading='auto')
ax5.pcolormesh(X, Y, data_v[time//2], shading='auto')
ax6.pcolormesh(X, Y, data_v[time-1], shading='auto')

plt.show()

What do I want from the simulation?
I want to be able to change the parameters rather easily. Or display a bunch of plots 
What parameters do I have?

def plot_simulation(a,b):
